In [1]:
import networkx as nx 
import msgspec
from util import *
from qng import * 
# from business_class import *
from ipysigma import Sigma
# from address_util import *
import json
import duckdb as d
import pandas as pd 

In [3]:
duck = d.connect('deseguys.duckdb', read_only=True)

In [5]:
sql = """ 
SELECT DISTINCT name FROM entities WHERE entity_type = 'company' and is_primary = 1
"""
company_names = duck.sql(sql).fetchall()

In [8]:
address_names = [] 
counter = 0 

for c in company_names:
    cn = c[0]
    clean = clean_street(cn)
    if clean != cn: 
        address_names.append(cn) 
        # print(cn) 
    # counter += 1
    # if counter == 1000:
        # break

In [ ]:
# sql = f"""
#                 SELECT DISTINCT 
#                     e.*, 
#                     pri.name as link_to, 
#                     pri.entity_type as primary_type
#                 FROM 
#                     entities e 
#                 JOIN 
#                     entities pri 
#                 ON 
#                     e.index_type = pri.index_type
#                     AND e.index_value = pri.index_value
#                     AND pri.is_primary = 1
#                 WHERE 
#                     pri.entity_type = 'committee'
#                     AND pri.name like '{name}'
#                     ORDER BY e.entity_type asc
#     """
# name = "FRIENDS OF GEORGE %CARDENAS" 
# sql = base_sql + f" AND pri.name like '{name}' ORDER BY e.entity_type asc;"
# len(duck.execute(sql).fetchall())

In [ ]:
# duck.sql("""
# SET @test = (SELECT max(id) FROM entities WHERE is_primary=1 AND name like 'FRIENDS OF GEORGE %CARDENAS' and entity_type == 'committee');")

In [2]:
def load_schema(filename:str):
    with open(filename, 'r') as f:
        schema = msgspec.json.decode(f.read(), type=GraphSchema)
    return schema

def load_graph(filename:str):
     with open(filename, 'r') as f:
        return msgspec.json.decode(f.read(), type=QNG)
    
def save_graph(filename:str, graph): 
    with open(filename, 'wb') as f:
        f.write(graph)

In [3]:
graph_schema = load_schema('graph_schemas/entity.qngs')
graph_factory = GraphFactory(
    node_factories= list(dict(graph_schema.node_factories).values()), 
    link_factories= graph_schema.link_factories
)


G = nx.MultiDiGraph()
# G = graph_entities(G, graph_factory, entities, [])

In [ ]:

G = load_graph('/Users/AnthonyMoser/Dev/public_data_tools/shiny/deseguys/app/quick_network_graph(37).qng').multigraph()

In [ ]:
def expand_nodes(G, nodes:list, search_types: list = ['companies', 'contracts', 'campaigns'], already_imported:list = [])->dict:
    entity_types = {
        "companies": ["company"],
        "contracts": ["contract"], 
        "campaigns": ["committee", "donor"]
    }
    included_types = []
    for st in search_types:
        included_types.extend(entity_types[st])
    included_types = f"{str(included_types)[1:-1]}"
    
    streets = [] 
    names = []
    companies = []
    for n in nodes:
        try:
            match G.nodes(data="type")[n]:
                case "address":
                    streets.append(n)
                case "company": 
                    companies.append(f"{G.nodes[n]['index_type']}-{G.nodes[n]['index_value']}") 
                case "committee": 
                    pass 
                case _: 
                    names.append(n) 
            
        except Exception as e:
            print(e)
            print(n)
            print(G.nodes(n))
            continue
    sf = pd.DataFrame(data = streets, columns=['street']).drop_duplicates()
    nf = pd.DataFrame(data = names, columns = ["name"]).drop_duplicates()
    cf = pd.DataFrame(data = companies, columns = ["idx"]).drop_duplicates()
    aie = pd.DataFrame(data = already_imported, columns = ["already_imported"])
    
    results = duck.sql(f""" 
        SELECT  
            e.*, 
            pri.name as link_to, 
            pri.entity_type as primary_type 
        FROM 
            entities e
        JOIN 
            entities pri 
        ON 
            e.index_type = pri.index_type
            AND e.index_value = pri.index_value
            AND pri.is_primary = 1
        WHERE
            pri.entity_type IN ({included_types})
            AND e.id not in (SELECT already_imported FROM aie) 
            AND (
                e.name in (SELECT name FROM nf) 
                OR e.street IN (SELECT street FROM ef)
                OR CONCAT(e.index_type, '-', e.index_value) IN (SELECT idx FROM cf)
            )
    """).fetchall()
    entities = {}
    for r in results:
        e = Entity(*r)
        entities[e.id] = e 
    return entities

In [ ]:
def get_entities_by_indexes(index_type: str, index_values :list, is_primary = 0):
    results = duck.sql(f"""
        SELECT 
            e.*, 
            pri.name as link_to, 
            pri.entity_type as primary_type
        FROM 
            entities e 
        JOIN 
            entities pri 
        ON 
            e.index_type = pri.index_type
            AND e.index_value = pri.index_value
            AND pri.is_primary = 1
        WHERE 
            e.index_type = '{index_type}'
            AND e.index_value in ({str(index_values)[1:-1]})
            AND e.is_primary = {is_primary}
        """
    ).fetchall() 

In [ ]:

def get_linked_entities(entities:dict, already_imported:list = []):
    linked_entities = {}
    both_searches = []
    primary_searches = []
    
    for entity in entities:
        e = entities[entity]
        try:
            if e.primary_type in ['company', 'contract']:
                both_searches.append(f"{e.index_type}-{e.index_value}")
            else:
                primary_searches.append(f"{e.index_type}-{e.index_value}")
        except Exception as ex:
            print(ex)
            print(e)
    bs = pd.DataFrame(data=both_searches, columns=['idx']).drop_duplicates()
    ps = pd.DataFrame(data=primary_searches, columns=['idx']).drop_duplicates()
    aie = pd.DataFrame(data=already_imported, columns = ['already_imported'])
    
    both_results = duck.sql(f""" 
        SELECT DISTINCT 
            e.*, 
            pri.name as link_to, 
            pri.entity_type as primary_type 
        FROM 
            entities e
        JOIN 
            entities pri 
        ON 
            e.index_type = pri.index_type
            AND e.index_value = pri.index_value
            AND pri.is_primary = 1
        WHERE
            e.id not in (SELECT already_imported FROM aie) 
            AND CONCAT(e.index_type, '-', e.index_value) IN (SELECT idx FROM bs)
    """).fetchall()
    
    primary_results = duck.sql(f""" 
        SELECT DISTINCT 
            e.*, 
            pri.name as link_to, 
            pri.entity_type as primary_type 
        FROM 
            entities e
        JOIN 
            entities pri 
        ON 
            e.index_type = pri.index_type
            AND e.index_value = pri.index_value
            AND pri.is_primary = 1
        WHERE
            e.id not in (SELECT already_imported FROM aie) 
            AND CONCAT(e.index_type, '-', e.index_value) IN (SELECT idx FROM bs)
            AND e.is_primary = 1
    """).fetchall()
    
    linked_entities = {}
    for r in both_results:
        e = Entity(*r)
        linked_entities[e.id] = e 
    
    for r in primary_results:
        e = Entity(*r)
        linked_entities[e.id] = e 
        
    return linked_entities

In [ ]:
entities = get_entities(name="FRIENDS OF GEORGE A. CARDENAS", street=None, file_number = None, search_types=['campaigns'], manual_override=True)

In [ ]:
linked_entities = get_linked_entities(entities)

In [ ]:
exp_entities = expand_nodes(G, list(G.nodes()))

In [ ]:
len(G)

In [ ]:
detailed = [ n[0] for n in G.nodes(data='details', default=None) if n[1] is not None]
detailed

In [ ]:
G.nodes['OGDEN CONST & RELIABLE ASPHALT']	

In [4]:
# G = load_graph('asphalt companies with city contracts.qng').multigraph()
G = load_graph('quick_network_graph(37).qng').multigraph()
def parse_amounts(G):
    amount_fields = {
        "contract": "award_amount", 
        "donation": "amount"
    }

    for u, v in G.edges():
        links = G[u][v]
        for link in links:
            details_string = G[u][v][link].get('details', '{}')
            details = json.loads(details_string)
            amount_field = amount_fields.get(G[u][v][link]['type'], "no_amount")
            link_size = int(float(details.get(amount_field, 1)))
            G[u][v][link]['size'] = link_size
            G.nodes[v]['size'] = G.nodes[v].get('size', 1) + link_size
            G.nodes[u]['size'] = G.nodes[u].get('size', 1) + link_size
    return G
    

In [5]:
G1 = parse_amounts(G)

In [ ]:
# large_layout = {
#         "adjustSizes": False,
#         # "barnesHutOptimize": True,
#         # "barnesHutTheta":3,
#         "linLogMode" :False,
#         "StrongGravityMode": True,
#         "edgeWeightInfluence":5,
#         "gravity":.5,
#         "outboundAttractionDistribution": False, 
#         "scalingRatio": .5
# }
# SF = SigmaFactory(layout_settings=large_layout, clickable_edges=True, edge_size='size', node_size='size', node_size_range=(3,40))
# viz = SF.make_sigma(G1) 
# viz

In [27]:
large_layout = {
        # "adjustSizes": True,
        # "barnesHutOptimize": False,
        # "barnesHutTheta":1.2,
        # "StrongGravityMode": False,
        # "edgeWeightInfluence":.5,
        # "scalingRatio": .5, 
        "gravity":.2,
        "outboundAttractionDistribution": True
}
Sigma(
        G1, 
        # height = self.height,
        layout_settings = large_layout, 
        node_size =             'size',
        edge_size =             'size',
        # node_size_scale =       ("pow", 2),
        # edge_size_scale =       'log+1',
        # edge_weight =           'size',
        edge_size_range=        (1, 15),
        # default_edge_color =    self.default_edge_color,
        clickable_edges =       True,
        # camera_state =          self.camera_state if len(camera_state) == 0 else camera_state,
        # node_size =             self.node_size if self.node_size else G.degree,
        # node_size_range =       self.node_size_range, 
        node_color =            "type",
        # node_color_palette=     node_colors,
        # selected_node=          self.selected_node,
        # layout =                self.layout if layout is None else layout,
        raw_node_border_color = "#FFFFFF",
        raw_node_border_ratio =  ".01",
        node_zindex =           lambda n: 1/G.nodes[n]['size'], 
        start_layout =          (len(G1) / 10 )
) 

Sigma(nx.MultiDiGraph with 2,698 nodes and 5,239 edges)